# [CPSC 322](https://github.com/GonzagaCPSC322) Data Science Algorithms
[Gonzaga University](https://www.gonzaga.edu/)

[Gina Sprint](http://cs.gonzaga.edu/faculty/sprint/)

# Model Deployment
What are our learning objectives for this lesson?
* Create our own Docker container image for a Flask app
* Deploy the Docker container image to Heroku

Content used in this lesson is based upon information in the following sources:
* [Heroku Deploying with Docker](https://devcenter.heroku.com/categories/deploying-with-docker)

## Creating Docker Container Images
Docker containers are instances of Docker images. We have been using the [continuumio/anaconda3](https://hub.docker.com/r/continuumio/anaconda3) Docker image and a container based on this image as our Python development environment in this class. We are now going to learn how to make our own Docker images, and then deploy a container based on this image to the web. Let's get started!

Build specifications for an image are listed in a special file called `Dockerfile`. From the [Docker Docs](https://docs.docker.com/engine/reference/builder/):
>Docker can build images automatically by reading the instructions from a Dockerfile. A Dockerfile is a text document that contains all the commands a user could call on the command line to assemble an image. Using docker build users can create an automated build that executes several command-line instructions in succession.

Here is an example `Dockerfile` for building a Flask image:

```docker
FROM continuumio/anaconda3:2020.11

ADD . /code
WORKDIR /code
ENTRYPOINT ["python", "app.py"]
```

Here is what each part of this `Dockerfile` specifies about the to-be-built image:
* `FROM <image>[:<tag>]`: A Dockerfile must begin with a FROM instruction. The FROM instruction specifies the Parent Image from which you are building.
    * For our Flask app, our base image will be the same [continuumio/anaconda3](https://hub.docker.com/r/continuumio/anaconda3) Docker image we have been using in this class.
* `ADD <src> <dest>`: The ADD instruction copies new files, directories or remote file URLs from `<src>` and adds them to the filesystem of the image at the path `<dest>`.
    * For our Flask app, we will copy the contents of the current working directory (which should be our project directory which has our `app.py` in it) into a new `/code` directory of the container when a container is instantiated.
* `WORKDIR /path/to/workdir`: The WORKDIR instruction sets the working directory for any RUN, CMD, ENTRYPOINT, COPY and ADD instructions that follow it in the Dockerfile. If the WORKDIR doesn’t exist, it will be created even if it’s not used in any subsequent Dockerfile instruction.
    * For our Flask app, we want our current working directory of a container to be `/home` so our next command (`ENTRYPOINT`) can use a simple relative path to refer to `app.py`.
* `ENTRYPOINT ["executable", "param1", "param2"]`: An ENTRYPOINT allows you to configure a container that will run as an executable. Note: You can use the exec form of ENTRYPOINT to set fairly stable default commands and arguments and then use either form of CMD to set additional defaults that are more likely to be changed.
    * For our Flask app, our container is an executable that runs the Flask app. We will invoke the Flask app the same we did directly when we were debugging it.
    
To create the image, cd into the project directory with the `Dockerfile` (typically this is in your project's top-level directory) and run `docker build -t flaskapp/interview:latest .` On success, your output should look like this:

```bash
sprint@cps-25626 APIServiceFun % docker build -t flaskapp/interview:latest .
[+] Building 0.2s (8/8) FINISHED                                                                           
 => [internal] load build definition from Dockerfile                                                  0.0s
 => => transferring dockerfile: 219B                                                                  0.0s
 => [internal] load .dockerignore                                                                     0.0s
 => => transferring context: 2B                                                                       0.0s
 => [internal] load metadata for docker.io/continuumio/anaconda3:2020.11                              0.0s
 => [internal] load build context                                                                     0.0s
 => => transferring context: 703B                                                                     0.0s
 => [1/3] FROM docker.io/continuumio/anaconda3:2020.11                                                0.0s
 => CACHED [2/3] ADD . /code                                                                          0.0s
 => CACHED [3/3] WORKDIR /code                                                                        0.0s
 => exporting to image                                                                                0.0s
 => => exporting layers                                                                               0.0s
 => => writing image sha256:519113531d754a411acdd4b18411dd5d89fd61257c3f9df46aa9890f862feac5          0.0s
 => => naming to docker.io/flaskapp/interview:latest   
```

To confirm the image was created, run `docker images`. You can also open Docker Desktop and click on Images. Now, you can create a container from this image with the `docker run` command just like we did with the [continuumio/anaconda3](https://hub.docker.com/r/continuumio/anaconda3) Docker image!! How cool!!

## Deploying a Container Image to Heroku
[Heroku](https://www.heroku.com) is a platform as a service (PaaS) that enables developers to build, run, and operate applications entirely in the cloud. We will use Heroku to host our Flask Docker container on the web so anyone can use it!

Heroku provides two ways for you to deploy your app with Docker:
1. Container Registry allows you to deploy pre-built Docker images to Heroku
    * Instructions: https://devcenter.heroku.com/articles/container-registry-and-runtime)
    * This approach requires you to install the Heroku CLI on your host machine
1. Build your Docker images with heroku.yml for deployment to Heroku
    * Instructions: https://devcenter.heroku.com/articles/build-docker-images-heroku-yml
    * This approach does not require you to install the Heroku CLI on your host machine. Instead, if you wish, you can install the Heroku CLI in our anaconda3_cpsc322 container.
1. (Unofficial #3)... Use a Github Action, such as [Build, Push and Release a Docker container to Heroku](https://github.com/marketplace/actions/build-push-and-release-a-docker-container-to-heroku)

Shared initial steps for both approaches:
1. Make sure your Flask `run()` command has the following arguments (read more about this [here](https://medium.com/@ksashok/containerise-your-python-flask-using-docker-and-deploy-it-onto-heroku-a0b48d025e43)):
```python
port = int(os.environ.get("PORT", 5000))
app.run(host='0.0.0.0', port=port, debug=False)
```
1. Create a Heroku account
1. Install the Heroku CLI. 
    1. For approach #1, install it on your host machine. 
        * On Mac: `brew tap heroku/brew && brew install heroku`
        * On Windows: Download the appropriate Windows installer [here](https://devcenter.heroku.com/articles/heroku-cli)
    1. For approach #2, install it in the anaconda3_cpsc322 Docker container:
        1. Install `gnupg`: `apt install gnupg`
        1. Install the Heroku CLI: `curl https://cli-assets.heroku.com/install-ubuntu.sh | sh`
1. Login to Heroku via the CLI: `heroku login -i` (then enter your Heroku email and password)

### Approach #1: Deploy Pre-built Docker Images
1. Go to https://dashboard.heroku.com/apps and create a new Heroku app, then choose "Container Registry Use Heroku CLI"
1. Back at the command line, log in to Container Registry: `heroku container:login`
1. Then build and image and push it to Heroku's Container Registry: `heroku container:push web`
1. Release the newly pushed images to deploy your app: `heroku container:release web`

### Approach #2: Build your Docker Images on Heroku
1. If you have not already done so, make your top-level project directory into a local git repository (e.g. `git init`, `git add -A`, `git commit -m "initial"`)
1. In your project's top-level directory, create a Heroku app from your project: `heroku create <app name>`
    1. This should add a new remote to your local Git repo for Heroku (confirm with `git remote -v`)
1. Create a `heroku.yml` file with the following contents:
```yml
build:
  docker:
    web: Dockerfile
```
1. This specifies how to build the docker image. Commit this file to your local repo.
1. Set the stack of your app to container: `heroku stack:set container`
1. Push your app to Heroku: `git push heroku master`

### (Unofficial) Approach #3: Set up Github Actions to Deploy Docker Images to Heroku
1. TBD